In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [14]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [19]:
size_train = train.shape[0]
dataset = pd.concat((train, test)).reset_index()

In [30]:
def print_features_missing(df):
    count = df.isnull().sum().to_dict()
    dict_null = {k: v for k, v in count.items() if v > 0}
    for k, v in dict_null.items():
        print(k, ' ' * (20 - len(str(k))), v)

def get_features_complete(df):
    count = df.isnull().sum().to_dict()
    return [k for k, v in count.items() if v == 0]

In [56]:
y = train['SalePrice'].transform(np.log)
features = get_features_complete(dataset)
dataset = dataset[features]

In [59]:
categorical = dataset.describe(include='O').columns.tolist()
dataset = pd.get_dummies(dataset, categorical)

In [61]:
features_drop = ['index', 'Id']
X = dataset.iloc[:size_train].drop(features_drop, axis=1)
X_test = dataset.iloc[size_train:].drop(features_drop, axis=1)

In [63]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=100, max_depth=5)
model.fit(X, y)
y_pred = np.exp(model.predict(X_test))

In [69]:
output = pd.DataFrame({'Id': dataset.iloc[size_train:, 1], 'SalePrice': y_pred})
output.to_csv('data/submission_0808_gboost.csv', index=False)